In [1]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 29kB/s 


In [2]:
!git clone https://github.com/D2DOnline/FaceRecognition1.git

Cloning into 'FaceRecognition1'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 90 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (90/90), done.


In [3]:
!ls FaceRecognition1


data-preprocess.py  main.ipynb	README.md  test-images	train-images


In [4]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

Using TensorFlow backend.


In [5]:
def extract_face(filename, required_size=(160, 160)):
	image = Image.open(filename)
	image = image.convert('RGB')
	pixels = asarray(image)
	detector = MTCNN()
	results = detector.detect_faces(pixels)
	x1, y1, width, height = results[0]['box']
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	face = pixels[y1:y2, x1:x2]
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

In [6]:
def load_faces(directory):
	faces = list()
	for filename in listdir(directory):
		path = directory + '/' + filename
		face = extract_face(path)
		faces.append(face)
	return faces

In [7]:
def load_dataset(directory):
	X, y = list(), list()
	for subdir in listdir(directory):
		path = directory + '/' + subdir
		if not isdir(path):
			continue
		faces = load_faces(path)
		labels = [subdir for _ in range(len(faces))]
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)


In [8]:
trainX, trainY = load_dataset('FaceRecognition1/train-images')
print(trainX.shape, trainY.shape)
testX, testY = load_dataset('FaceRecognition1/test-images')
savez_compressed('FaceRecognition1/face-dataset.npz', trainX, trainY, testX, testY)

>loaded 8 examples for class: anand
>loaded 8 examples for class: guru
>loaded 8 examples for class: jitendra
(24, 160, 160, 3) (24,)
>loaded 4 examples for class: anand
>loaded 3 examples for class: guru
>loaded 3 examples for class: jitendra


In [9]:
!ls FaceRecognition1

data-preprocess.py  main.ipynb	test-images
face-dataset.npz    README.md	train-images
